In [ ]:
# Conversation Management & Classification using Groq API

#Assignment Submission by: Sehba Hanief

!pip install --upgrade --quiet groq jsonschema
import os


os.environ["GROQ_API_KEY"] = input("🔑 Enter your GROQ API key: ")




In [13]:
from groq import Groq
client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [14]:
chat_completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Hello Groq, can you hear me?"}]
)

print(chat_completion.choices[0].message)


ChatCompletionMessage(content="However, I need to clarify that I'm not Groq, but rather an AI assistant. I'm here to help answer your questions and provide information. If you're referring to Groq, the company is likely a tech firm known for its deep learning processor designs.", role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None)


In [16]:
class ConversationHistory:
    def __init__(self, client, summarization_model="llama-3.1-8b-instant"):
        self.client = client
        self.messages = []
        self.run_count = 0
        self.summarization_model = summarization_model

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})
        self.run_count += 1

    def truncate_by_turns(self, n):
        return self.messages[-n*2:]

    def truncate_by_chars(self, max_chars):
        result, count = [], 0
        for msg in reversed(self.messages):
            if count + len(msg['content']) > max_chars:
                break
            result.insert(0, msg)
            count += len(msg['content'])
        return result

    def summarize(self, window_size=6):
        window = self.messages[-window_size:]
        content = "\n".join([f"{m['role']}: {m['content']}" for m in window])
        response = self.client.chat.completions.create(
            model=self.summarization_model,
            messages=[
                {"role": "system", "content": "Summarize this conversation briefly."},
                {"role": "user", "content": content}
            ]
        )
        return response.choices[0].message.content

    def periodic_summarize_and_replace(self, k=3, window_size=6):
        if self.run_count % k == 0 and len(self.messages) >= window_size:
            summary = self.summarize(window_size)
            self.messages = self.messages[:-window_size] + [
                {"role": "system", "content": f"[Summary]: {summary}"}
            ]

    def get_history(self):
        return self.messages


In [17]:
history = ConversationHistory(client)

# Simulate a chat
samples = [
    ("user", "Hi, I am John from Delhi."),
    ("assistant", "Hello John, how can I help you?"),
    ("user", "I am 25 years old and my email is john@example.com."),
    ("assistant", "Thanks for sharing your details."),
    ("user", "My phone number is 9876543210."),
    ("assistant", "Got it, John."),
]

for role, content in samples:
    history.add_message(role, content)
    history.periodic_summarize_and_replace(k=3)

print("=== Full History ===")
print(history.get_history())

print("\n=== Truncated by last 2 turns ===")
print(history.truncate_by_turns(2))

print("\n=== Truncated by 80 chars ===")
print(history.truncate_by_chars(80))


=== Full History ===
[{'role': 'system', 'content': '[Summary]: There is no conversation to summarize as the conversation only consisted of you playing the role of John sharing your details with me, and I responding accordingly.'}]

=== Truncated by last 2 turns ===
[{'role': 'system', 'content': '[Summary]: There is no conversation to summarize as the conversation only consisted of you playing the role of John sharing your details with me, and I responding accordingly.'}]

=== Truncated by 80 chars ===
[]


In [20]:
import json
from jsonschema import validate

EXTRACTION_SCHEMA = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "string"},
    },
    "required": ["name", "email", "phone", "location", "age"]
}

def extract_info(chat_text):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system",
                "content": "Extract the following fields and return them strictly as JSON: "
                           "name, email, phone, location, age."
            },
            {
                "role": "user",
                "content": f"Chat: {chat_text}\n\nReturn only valid JSON with those fields."
            }
        ],
        response_format={"type": "json_object"}  # forces JSON output
    )
    return json.loads(response.choices[0].message.content)


def validate_extraction(data):
    try:
        validate(instance=data, schema=EXTRACTION_SCHEMA)
        return True
    except Exception as e:
        return str(e)


In [21]:
sample_chats = [
    "Hi, I am John Doe from Delhi, 25 years old. My email is john@example.com and my phone is 9876543210.",
    "My name is Sarah Khan, 30 years old, living in Mumbai. Contact me at sarah@gmail.com, phone 9123456789.",
    "Hello, this is Mike from Bangalore, age 22. You can reach me at mike22@yahoo.com, phone number 9988776655."
]

for chat in sample_chats:
    extracted = extract_info(chat)
    print("\nChat:", chat)
    print("Extracted:", extracted)
    print("Validation:", validate_extraction(extracted))



Chat: Hi, I am John Doe from Delhi, 25 years old. My email is john@example.com and my phone is 9876543210.
Extracted: {'name': 'John Doe', 'email': 'john@example.com', 'phone': '9876543210', 'location': 'Delhi', 'age': 25}
Validation: 25 is not of type 'string'

Failed validating 'type' in schema['properties']['age']:
    {'type': 'string'}

On instance['age']:
    25

Chat: My name is Sarah Khan, 30 years old, living in Mumbai. Contact me at sarah@gmail.com, phone 9123456789.
Extracted: {'name': 'Sarah Khan', 'email': 'sarah@gmail.com', 'phone': '9123456789', 'location': 'Mumbai', 'age': 30}
Validation: 30 is not of type 'string'

Failed validating 'type' in schema['properties']['age']:
    {'type': 'string'}

On instance['age']:
    30

Chat: Hello, this is Mike from Bangalore, age 22. You can reach me at mike22@yahoo.com, phone number 9988776655.
Extracted: {'name': 'Mike', 'email': 'mike22@yahoo.com', 'phone': '9988776655', 'location': 'Bangalore', 'age': '22'}
Validation: True
